# Kindle Note Caster


### problématique:

kindle fournit un csv ou bien un pdf, pas pratique pour relire ses notes et en faire un seul document facile à lire: je les cast en Microsoft word

### détails techniques:

https://anaconda.org/conda-forge/python-docx

### améliorations possibles:

annotation + mettre le nom du titre du livre en haut

gestion du chemin absolu ( pour l'instant doit etre dans le dossier)

gestion différamment des surlignements et des notes

In [1]:
### import

import pandas as pd
import numpy as np
import docx

In [2]:
#### Nom des chemins:

i_chemin_du_csv= "A.csv"
o_chemin_du_word= "The_Customer_Success_Professionals_Handbook.docx"


In [3]:
### script
df=pd.read_csv(i_chemin_du_csv)
df

,Vos notes Kindle pour :,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,THE CUSTOMER SUCCESS PROFESSIONAL'S HANDBOOK: ...,NaN,NaN,NaN
1,"de Ashvin Vaidyanathan, Ruben Rabago",NaN,NaN,NaN
2,Aperçu instantané Kindle gratuit :,NaN,NaN,NaN
3,https://amzn.eu/9RsToTG,NaN,NaN,NaN
4,----------------------------------------------,NaN,NaN,NaN
...,...,...,...,...
256,Surlignement (Jaune),Emplacement 4376,NaN,Depending on the needs of each customer segmen...
257,Surlignement (Jaune),Emplacement 4410,NaN,SMB Customer Success Manager versus an Enterpr...
258,Surlignement (Jaune),Emplacement 4426,NaN,it is essential to develop two separate career...
259,Surlignement (Jaune),Emplacement 4564,NaN,common framework that is still being used for ...


In [4]:
o_texte = ""
List_mot_a_enlever=['nan','Annotation']

def concat_text(text):
    global o_texte
    if (str(text) not in List_mot_a_enlever):
        o_texte = o_texte +'\n\n'+ str(text)

In [5]:
df['Unnamed: 3'].apply(concat_text)

0      None
1      None
2      None
3      None
4      None
       ... 
256    None
257    None
258    None
259    None
260    None
Name: Unnamed: 3, Length: 261, dtype: object

In [6]:
mydoc = docx.Document()
mydoc.add_paragraph(o_texte)
mydoc.save(o_chemin_du_word)
